이 코드는 Public score 33위에서 Private score 4위로 오른 코드입니다. 제가 어떻게 코드를 짰고 왜 그렇게 짰는지를 이 노트북에 설명해놨습니다. LB에만 집중하기보다 cv에도 집중을 한 게 좋은 성적을 낸 이유가 아닐까 싶습니다.  

이 노트북에는 cv를 다루지 않았습니다. cv는 다른 노트북으로 테스트를 했고 여기서는 최종 모델에 대한 코드만 담았습니다.

# 라이브러리 호출 및 I/O

In [1]:
# 데이터 분석 라이브러리
import numpy as np
import pandas as pd

# 시각화 라이브러리
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 모델링 라이브러리
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

# 기타 라이브러리
import random
import gc
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
sns.set_style("whitegrid")

/kaggle/input/kakr-4th-competition/sample_submission.csv
/kaggle/input/kakr-4th-competition/test.csv
/kaggle/input/kakr-4th-competition/train.csv


In [2]:
train = pd.read_csv('/kaggle/input/kakr-4th-competition/train.csv')
test = pd.read_csv('/kaggle/input/kakr-4th-competition/test.csv')
sample_submission = pd.read_csv('/kaggle/input/kakr-4th-competition/sample_submission.csv')

# 데이터 간단히 둘러보기

- 우선 데이터를 전반적으로 살펴보겠습니다.

In [3]:
train.shape, test.shape, sample_submission.shape

((26049, 16), (6512, 15), (6512, 2))

- train 데이터에는 26,049개의 데이터가 존재하고, test 데이터에는 6,512개의 데이터가 존재합니다.
- train 데이터에는 target variable도 포함하므로 test 데이터보다 feature 개수가 1개 더 많습니다.

In [4]:
train.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K


id를 제외하고 총 15개의 feature가 있으며 각 feature는 다음과 같은 의미를 가지고 있습니다. (data description에서 제공)

- `age` : 나이
- `workclass` : 고용 형태
- `fnlwgt` : 사람 대표성을 나타내는 가중치 (final weight의 약자)
- `education` : 교육 수준 (최종 학력)
- `education_num` : 교육 수준 수치
- `marital_status`: 결혼 상태
- `occupation` : 업종
- `relationship` : 가족 관계
- `race` : 인종
- `sex` : 성별
- `capital_gain` : 양도 소득
- `capital_loss` : 양도 손실
- `hours_per_week` : 주당 근무 시간
- `native_country` : 국적
- `income` : **수익 (예측해야 하는 값, target variable)**

In [5]:
test.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,0,28,Private,67661,Some-college,10,Never-married,Adm-clerical,Other-relative,White,Female,0,0,40,United-States
1,1,40,Self-emp-inc,37869,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
2,2,20,Private,109952,Some-college,10,Never-married,Handlers-cleaners,Own-child,White,Male,0,0,25,United-States
3,3,40,Private,114537,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States
4,4,37,Private,51264,Doctorate,16,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,99,France


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26049 entries, 0 to 26048
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26049 non-null  int64 
 1   age             26049 non-null  int64 
 2   workclass       26049 non-null  object
 3   fnlwgt          26049 non-null  int64 
 4   education       26049 non-null  object
 5   education_num   26049 non-null  int64 
 6   marital_status  26049 non-null  object
 7   occupation      26049 non-null  object
 8   relationship    26049 non-null  object
 9   race            26049 non-null  object
 10  sex             26049 non-null  object
 11  capital_gain    26049 non-null  int64 
 12  capital_loss    26049 non-null  int64 
 13  hours_per_week  26049 non-null  int64 
 14  native_country  26049 non-null  object
 15  income          26049 non-null  object
dtypes: int64(7), object(9)
memory usage: 3.2+ MB


- id를 제외하고 feature의 타입(type)은 int 6개, object 9개입니다. 
- object 타입은 string 타입이라고 보시면 됩니다.

In [7]:
train.describe()

,id,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,26049.000000,26049.000000,2.604900e+04,26049.000000,26049.00000,26049.000000,26049.000000
mean,13024.000000,38.569235,1.903045e+05,10.088372,1087.68970,87.732734,40.443126
std,7519.842917,13.671489,1.059663e+05,2.567610,7388.85469,403.230205,12.361850
min,0.000000,17.000000,1.376900e+04,1.000000,0.00000,0.000000,1.000000
25%,6512.000000,28.000000,1.181080e+05,9.000000,0.00000,0.000000,40.000000
50%,13024.000000,37.000000,1.788660e+05,10.000000,0.00000,0.000000,40.000000
75%,19536.000000,48.000000,2.377350e+05,12.000000,0.00000,0.000000,45.000000
max,26048.000000,90.000000,1.484705e+06,16.000000,99999.00000,4356.000000,99.000000


In [8]:
train.describe(include='O')

,workclass,education,marital_status,occupation,relationship,race,sex,native_country,income
count,26049,26049,26049,26049,26049,26049,26049,26049,26049
unique,9,16,7,15,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,18123,8433,11970,3304,10541,22315,17482,23371,19744


In [9]:
print(f'income count: {train["income"].count()}')
print(f'income not null count(pct): {np.round(train["income"].count()/len(train)*100, 2)}%')

income count: 26049
income not null count(pct): 100.0%


## Feature별 Null 값 확인

In [10]:
num_of_null = train.isnull().sum()
percent = (train.isnull().sum() / train.isnull().count() * 100)
pd.concat([num_of_null, percent], axis=1, keys=['# of null', 'Percent']).sort_values(by='Percent', ascending=False)

,# of null,Percent
id,0,0.0
age,0,0.0
workclass,0,0.0
fnlwgt,0,0.0
education,0,0.0
education_num,0,0.0
marital_status,0,0.0
occupation,0,0.0
relationship,0,0.0
race,0,0.0


- null값이 하나도 존재하지 않는다는 것을 알 수 있습니다.

In [11]:
num_of_null = test.isnull().sum()
percent = (test.isnull().sum() / test.isnull().count() * 100)
pd.concat([num_of_null, percent], axis=1, keys=['# of null', 'Percent']).sort_values(by='Percent', ascending=False)

,# of null,Percent
id,0,0.0
age,0,0.0
workclass,0,0.0
fnlwgt,0,0.0
education,0,0.0
education_num,0,0.0
marital_status,0,0.0
occupation,0,0.0
relationship,0,0.0
race,0,0.0


- test 데이터도 마찬가지로 null값이 하나도 없습니다.

# Feature Engineering

Feature Engineering은 [Teddy Lee님의 notebook](https://www.kaggle.com/teddylee777/eda-lightgbm-pycaret)을 참고했습니다.

education을 총 8개의 카테고리로 나누었습니다. 분포도를 살펴봤을 때 아래와 같이 나누었을 때 성능이 가장 좋을 것으로 예상했습니다.


**분포도가 궁금하신 분은 [EDA notebook](https://www.kaggle.com/werooring/basic-eda-lgbm-modeling-public-score-0-87714#%EC%88%98%EC%B9%98%ED%98%95-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D)의 수치형 데이터 분석 - 다변량(mulvariate) 데이터 시각화 - 3) 교육에 따른 소득 수준 (education_num vs income) 그래프 참고!**

In [12]:
education_map = {
    'Preschool': 'level_0', # Preschool은 모두 소득수준이 낮음, 추후 모델링시 참고
    '1st-4th': 'level_1', 
    '5th-6th': 'level_1', 
    '7th-8th': 'level_1', 
    '9th': 'level_2', 
    '10th': 'level_2', 
    '11th': 'level_2', 
    '12th': 'level_2', 
    'HS-grad': 'level_3', 
    'Some-college': 'level_3', 
    'Assoc-acdm': 'level_4', 
    'Assoc-voc': 'level_4', 
    'Bachelors': 'level_5', 
    'Masters': 'level_6', 
    'Prof-school': 'level_7', 
    'Doctorate': 'level_7',
}



train['education'] = train['education'].map(education_map)
test['education'] = test['education'].map(education_map)

In [13]:
income_01 = ['Jamaica',
 'Haiti',
 'Puerto-Rico',
 'Laos',
 'Thailand',
 'Ecuador',]

income_02 = ['Outlying-US(Guam-USVI-etc)',
 'Honduras',
 'Columbia',
 'Dominican-Republic',
 'Mexico',
 'Guatemala',
 'Portugal',
 'Trinadad&Tobago',
 'Nicaragua',
 'Peru',
 'Vietnam',
 'El-Salvador',]

income_03 = ['Poland',
 'Ireland',
 'South',
 'China',]

income_04 = [
    'United-States',
]
income_05 = [
 'Greece',
 'Scotland',
 'Cuba',
 'Hungary',
 'Hong',
 'Holand-Netherlands',
]
income_06 = [
 'Philippines',
 'Canada',
]
income_07 = [
 'England',
 'Germany',
]

income_08 = [
 'Italy',
 'India',
 'Japan',
 'France',
 'Yugoslavia',
 'Cambodia',
]

income_09 = [
 'Taiwan',
 'Iran',
]

income_other=['?', ]

def convert_country(x):
    if x in income_01:
        return 'income_01'
    elif x in income_02:
        return 'income_02'
    elif x in income_03:
        return 'income_03'
    elif x in income_04:
        return 'income_04'
    elif x in income_05:
        return 'income_05'
    elif x in income_06:
        return 'income_06'
    elif x in income_07:
        return 'income_07'
    elif x in income_08:
        return 'income_08'
    elif x in income_09:
        return 'income_09'
    else:
        return 'income_other'
    
train['country_bin'] = train['native_country'].apply(convert_country)
test['country_bin'] = test['native_country'].apply(convert_country)

In [14]:
train.loc[train['workclass'] == 'Without-pay', 'workclass'] = 'other'
train.loc[train['workclass'] == 'Never-worked', 'workclass'] = 'other'
test.loc[test['workclass'] == 'Without-pay', 'workclass'] = 'other'
test.loc[test['workclass'] == 'Never-worked', 'workclass'] = 'other'

## Feature Drop

카테고리 feature로 변경한 두 피처 `education_num`과 `native_country`는 drop했고, feature와 무관한 `id`도 삭제했습니다. `fnlwgt`는 분포도를 살펴봤을 때 소득 수준을 구분하는 데 도움이 되지 않는다고 판단해 삭제했습니다.  

**`fnlwgt` 분포도가 궁금하신 분은 [EDA notebook](https://www.kaggle.com/werooring/basic-eda-lgbm-modeling-public-score-0-87714#%EC%88%98%EC%B9%98%ED%98%95-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D)의 수치형 데이터 분석 - 다변량(mulvariate) 데이터 시각화 - 2) Final weight 따른 소득 수준 (fnlwgt vs income) 그래프 참고!**

In [15]:
# 필요없는 feature drop
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

# eudcation으로 변경되었으므로 education_num은 삭제
train.drop(['education_num'], axis=1, inplace=True)
test.drop(['education_num'], axis=1, inplace=True)

# country_bin으로 변경되었으므로 native_country는 삭제
train.drop(['native_country'], axis=1, inplace=True)
test.drop(['native_country'], axis=1, inplace=True)

# 필요없는 feature drop
train.drop(['fnlwgt'], axis=1, inplace=True)
test.drop(['fnlwgt'], axis=1, inplace=True)

# 모델링 (Modeling)

- train feature와 target value를 분리합니다.

In [16]:
y = train['income'] != '<=50K'
X = train.drop(['income'], axis=1)

- 범주형 데이터를 인코딩합니다.
- `LabelEncoder`와 `OrdinalEncoder`는 동일한 기능을 갖습니다. 하지만 `OrdinalEncoder`는 주로 (n_samples, n_features) 형태를 갖는 Feature에 사용되며, `LabelEncoder`는 주로 (n_samples,) 형태를 갖는 Target에 사용됩니다.   
(출처:  https://datascience.stackexchange.com/questions/39317/difference-between-ordinalencoder-and-labelencoder/64177)

In [17]:
# 범주형 데이터를 갖는 칼럼 가져오기
LE_columns = []

for col in X.columns:
    if X[col].dtype.name == 'object':
        LE_columns.append(col)
        
LE_columns       

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'country_bin']

In [18]:
# 라벨 인코더 생성
LE_encoder = OrdinalEncoder(LE_columns)

# train, test 데이터에 인코딩 적용
X = LE_encoder.fit_transform(X, y)
test = LE_encoder.transform(test)

Kfold = 7로 잡았습니다. 4, 5, 6, 7 중 테스트해봤을 때 cv 및 LB 성능이 종합적으로 높은 건 7이었습니다.  
또한 일반적으로 타겟값이 불균형일 때는 KFold보다 stratifiedKfold를 쓰지만 cv와 LB를 종합적으로 고려했을 때 KFold의 성능이 더 높게 나왔습니다. 이유는 명확히 모르겠으나 극심한 불균형이 아니라 그런 것 같습니다.

In [19]:
NFOLDS = 7
folds = KFold(n_splits=NFOLDS)

columns = X.columns
splits = folds.split(X, y)
y_preds = np.zeros(test.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = columns

여러 피처엔지니어링을 시도해봤으나 큰 성능 향상을 기대하지 못했습니다. 피처엔지니어링은 거의 안한만큼 하이퍼파라미터 튜닝에 조금 더 신경썼습니다. 저는 시간이 많지고 않았고 피처엔지니어링에 대한 아이디어가 더 이상 떠오르지 않아 어쩔 수 없이 하이퍼파라미터 튜닝에 시간을 좀 투자했습니다. 실제 성능향상에는 피처엔지니어링에 공을 들이는 것이 좋습니다.


n_estimator, learing_rate, num_leaves, min_child_samples, max_depth, reg_alpha, reg_lambda, colsample_bytree, subsample에 대해 튜닝했습니다. 우선 큰 범우로 GridSearch를 했고 점차 성능 좋은 범위로 좁혀가며 진행했습니다. cv와 LB를 종합적으로 고려했을 때 가장 좋은 성능이 나오는 하이퍼파라미터를 찾아내어 아래와 같이 사용했습니다.

In [20]:
model = LGBMClassifier(objective='binary', n_estimators=1000, learning_rate=0.01, # 기본 튜닝 (learning_rate은 작게하며 n_estimators를 키우는 것이 기본 원칙!)
                       num_leaves=42, min_child_samples=30, max_depth=30, # 과적합 방지
                       reg_alpha=0.3, reg_lambda=0.7, # regularization을 통한 과적합 방지
                       colsample_bytree=0.8, subsample=0.8, # 과적합 방지II
                       random_state=91)


for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold: ', fold_n+1)
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    evals = [(X_train, y_train), (X_valid, y_valid)]
    model.fit(X_train, y_train, eval_metric='f1', eval_set=evals, verbose=True)
    
    feature_importances[f'fold_{fold_n + 1}'] = model.feature_importances_
        
    y_preds += model.predict(test).astype(int) / NFOLDS
    
    del X_train, X_valid, y_train, y_valid
    gc.collect()

Fold:  1
[1]	training's binary_logloss: 0.549331	valid_1's binary_logloss: 0.546901
[2]	training's binary_logloss: 0.545099	valid_1's binary_logloss: 0.542766
[3]	training's binary_logloss: 0.541546	valid_1's binary_logloss: 0.539251
[4]	training's binary_logloss: 0.53735	valid_1's binary_logloss: 0.535159
[5]	training's binary_logloss: 0.533398	valid_1's binary_logloss: 0.531294
[6]	training's binary_logloss: 0.529402	valid_1's binary_logloss: 0.527355
[7]	training's binary_logloss: 0.525597	valid_1's binary_logloss: 0.523569
[8]	training's binary_logloss: 0.521943	valid_1's binary_logloss: 0.519941
[9]	training's binary_logloss: 0.518248	valid_1's binary_logloss: 0.516324
[10]	training's binary_logloss: 0.514636	valid_1's binary_logloss: 0.512806
[11]	training's binary_logloss: 0.511262	valid_1's binary_logloss: 0.509479
[12]	training's binary_logloss: 0.507814	valid_1's binary_logloss: 0.506111
[13]	training's binary_logloss: 0.504561	valid_1's binary_logloss: 0.502935
[14]	training

- feature_importance를 활용하여 각 폴드(fold)별로 피쳐 중요도를 살펴보겠습니다. 
- 단, feature_importance는 피쳐 중요도를 파악하기 위한 절대적인 방법이 아닙니다. 그냥 참고만 하시기 바랍니다. SHAP도 사용해봤는데 feature_importance와 결과가 상당히 달랐습니다. SHAP를 조금 더 신뢰하기 때문에 feature_importance는 거의 무시했습니다.

In [21]:
feature_importances

,feature,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7
0,age,8516,8958,8884,8626,8582,8626,8761
1,workclass,2439,2493,2449,2486,2351,2546,2588
2,education,4013,3895,4069,4036,3902,4082,4154
3,marital_status,1853,1889,1865,1972,1914,1801,1736
4,occupation,5740,5606,5715,5753,5978,5902,5960
5,relationship,2034,2076,2051,2190,2044,2161,2162
6,race,865,884,1044,865,846,823,900
7,sex,804,779,739,858,797,779,793
8,capital_gain,4542,4438,4364,4662,4563,4361,4254
9,capital_loss,3347,3138,2984,3159,3241,3327,3175


# 제출

In [22]:
sample_submission['prediction'] = y_preds

for ix, row in sample_submission.iterrows():
    if row['prediction'] > 0.5:
        sample_submission.loc[ix, 'prediction'] = 1
    else:
        sample_submission.loc[ix, 'prediction'] = 0

sample_submission = sample_submission.astype({"prediction": int})
sample_submission.to_csv('submission.csv', index=False)

- 마지막으로 submission의 결과를 살펴보겠습니다. 

In [23]:
sample_submission.head(10)

,id,prediction
0,0,0
1,1,1
2,2,0
3,3,1
4,4,1
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0
